In [ ]:
!!pip install accelerate==0.29.3 bitsandbytes==0.43.1 datasets==2.19.0 sentence-transformers==2.3.1 tokenizers==0.19.1 torch==2.2.1 transformers==4.40.1 trl==0.8.5

['Collecting accelerate==0.29.3',
 '  Downloading accelerate-0.29.3-py3-none-any.whl (297 kB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/297.6 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━\x1b \x1b204.8/297.6 kB\x1b \x1b5.9 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b297.6/297.6 kB\x1b \x1b5.8 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hCollecting bitsandbytes==0.43.1',
 '  Downloading bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl (119.8 MB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/119.8 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.6/119.8 MB\x1b \x1b18.1 MB/s\x1b eta \x1b0:00:07\x1b',
 '\x1b[2K     \x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b1.5/119.8 MB\x1b \x1b22.3 MB/s\x1b eta \x1b0:00:06\x1b',
 '\x1b[2K     \x1

```
accelerate==0.29.3
bitsandbytes==0.43.1
datasets==2.19.0
sentence-transformers==2.3.1
tokenizers==0.19.1
torch==2.2.1
transformers==4.40.1
trl==0.8.5
wandb==0.16.6
```

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Train

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained(
    "apple/OpenELM-3B-Instruct",
    trust_remote_code=True,
    device_map = {"": 0},
    torch_dtype = torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    use_fast=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments, set_seed, get_constant_schedule
from trl import SFTTrainer, setup_chat_format, DataCollatorForCompletionOnlyLM
from datasets import load_dataset
#import uuid, wandb

set_seed(42)
lr = 5e-5
#run_id = f"OpenELM-450M_LR-{lr}_OA_{str(uuid.uuid4())}"

model, tokenizer = setup_chat_format(model, tokenizer)
if tokenizer.pad_token in [None, tokenizer.eos_token]:
    tokenizer.pad_token = tokenizer.unk_token

dataset = load_dataset("elliotthwang/guanaco-llama2-chinese-1k")

training_arguments = TrainingArguments(
    output_dir = "./results",
    #evaluation_strategy = "steps",
    #label_names = ["labels"],
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 2,
    save_steps = -1,
    #save_steps = 250,
    #eval_steps = 250,
    logging_steps = 10,
    learning_rate = lr,
    num_train_epochs = 1,
    lr_scheduler_type = "constant",
    optim = 'paged_adamw_8bit',
    bf16 = False,
    #bf16 = True,
    gradient_checkpointing = True,
    group_by_length = True,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    #eval_dataset = dataset['test'],
    #data_collator = DataCollatorForCompletionOnlyLM(
    #max_seq_length = 2048,
    max_seq_length = 512,
    dataset_text_field="text",
    dataset_kwargs = dict(add_special_tokens = False),
    args = training_arguments,
    packing= False,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
#wandb.init(
#    project = "OpenELM",
#    name = run_id,
#).log_code(include_fn=lambda path: path.endswith(".py") or path.endswith(".ipynb"))
torch.tensor(train).to(torch.int64)
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,15.650000
20,14.975000
30,14.309400
40,13.784400
50,13.137500
60,13.131200
70,12.840600
80,12.362500
90,11.650000
100,11.309400


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

# Inference

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# change me!
model_path = "out/checkpoint-750"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    use_fast=False)

def prompt(question, debug=False, max_new_tokens=500):
    messages = [
        {"role": "user", "content": question},
    ]

    input_tokens = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")
    output_tokens = model.generate(input_tokens, max_new_tokens=max_new_tokens)

    if debug:
        for tok in output_tokens[0]:
            print(tok, tokenizer.decode(tok))

    output = tokenizer.decode(output_tokens[0], skip_special_tokens=False)

    print(output)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
prompt("hello, who are you?")

<|im_start|> user
hello, who are you? <|im_end|> 
 <|im_start|> assistant
I am Open Assistant, an open source chatbot developed by the Open Assistant Project. I am designed to be friendly, helpful, and informative. <|im_end|>
